In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame

In [2]:
#Upload the csv
disc_inp_data = pd.read_csv("C:\\Users\\sara1\\Desktop\\Workingdata\\discretised_input_data.csv", low_memory=False)

In [3]:
#Display the columns
disc_inp_data.head()

,stay_id,sofa_score,sirs,age,weight,gender,gcs,sofa_24hours,bilirubin_total,bicarbonate,...,albumin,creatinine,heart_rate,temperature,spo2,vaso_rate,vaso_amount,urineoutput,died_in_hosp,vaso_input
0,34712382,7,3,91,51.0,0,14,7,2.879823,36.0,...,2.895128,2.4,60.0,36.980675,96.0,2.400000,14.799999,0.0,0,2.0
1,34712382,7,3,91,51.0,0,14,7,2.879823,36.0,...,2.895128,2.4,60.0,36.980675,96.0,2.400000,14.799999,0.0,0,2.0
2,34712382,7,3,91,51.0,0,14,7,2.879823,36.0,...,2.895128,2.4,60.0,36.980675,96.0,2.400000,14.799999,0.0,1,2.0
3,34712382,7,3,91,51.0,0,14,7,2.879823,36.0,...,2.895128,2.4,60.0,36.980675,96.0,2.400000,14.799999,0.0,1,2.0
4,30902754,2,4,87,80.0,1,8,1,2.879823,36.0,...,2.895128,0.5,120.0,36.980675,97.0,2.406079,39.058680,30.0,1,4.0


In [4]:
#Find out how big the column is
disc_inp_data.shape

(6038365, 31)

In [5]:
#Show that no null values
print(disc_inp_data.isnull().sum())

stay_id                   0
sofa_score                0
sirs                      0
age                       0
weight                    0
gender                    0
gcs                       0
sofa_24hours              0
bilirubin_total           0
bicarbonate               0
hemoglobin                0
chloride                  0
calcium                   0
potassium                 0
sodium                    0
lactate                   0
glucose                   0
respiratory_rate_total    0
fio2                      0
platelet                  0
wbc                       0
albumin                   0
creatinine                0
heart_rate                0
temperature               0
spo2                      0
vaso_rate                 0
vaso_amount               0
urineoutput               0
died_in_hosp              0
vaso_input                0
dtype: int64


In [ ]:
#Goes through the columns in the dataframe except the first(stay_id) and last(died_in_hosp) to plot histograms in bins of 10
#for col in disc_inp_data.columns[1:-1]:
    # Extract the column values as a numpy array
#    col_values = disc_inp_data[col].values
    
    # Plot a histogram of the column values
#    plt.hist(col_values, bins=10)
#    plt.xlabel(col)
#    plt.ylabel('Frequency')
#    plt.title(f'Distribution of {col}')
#    plt.show()

In [ ]:
#for col in disc_inp_data.columns[1:-1]:
    # Extract the column values as a numpy array
#    col_values = disc_inp_data[col].values
    
    # Find the minimum and maximum values for the column
#    min_value = np.min(col_values)
#    max_value = np.max(col_values)
    
    # Print the minimum and maximum values for the column
#    print(f"Column '{col}': Minimum value = {min_value}, Maximum value = {max_value}")

In [6]:
# add rewards - sparsely for now; reward function shaping comes in a separate script
disc_inp_data['reward'] = 0
for i in disc_inp_data.index:
    if i == 0:
        continue
    else:
        if disc_inp_data.loc[i, 'stay_id'] != disc_inp_data.loc[i-1, 'stay_id']:
            if disc_inp_data.loc[i-1, 'died_in_hosp'] == 1:
                disc_inp_data.loc[i-1,'reward'] = -100
            elif disc_inp_data.loc[i-1, 'died_in_hosp'] == 0:
                disc_inp_data.loc[i-1,'reward'] = 100
            else:
                print ("error in row"), i-1
if disc_inp_data.loc[len(disc_inp_data)-1, 'died_in_hosp'] == 1:
    disc_inp_data.loc[len(disc_inp_data)-1, 'reward'] = -100
elif disc_inp_data.loc[len(disc_inp_data)-1, 'died_in_hosp'] == 0:
     disc_inp_data.loc[len(disc_inp_data)-1, 'reward'] = 100
print (disc_inp_data['reward'].value_counts())

 0      6035066
-100       1900
 100       1399
Name: reward, dtype: int64


In [7]:
#Replace the singular value in bicarbonate with random numbers from max and min val
column_name = 'bicarbonate'

# Generate random numbers within the specified range
min_value = 20
max_value = 36
random_numbers = np.random.randint(min_value, max_value + 1, size=len(disc_inp_data))

# Replace values in the column with random numbers
disc_inp_data[column_name] = random_numbers

In [8]:
# now split into train/validation/test sets
import random
unique_ids = disc_inp_data['stay_id'].unique()
random.shuffle(unique_ids)
train_sample = 0.7
val_sample = 0.1
test_sample = 0.2
train_num = int(len(unique_ids) * 0.7)
val_num = int(len(unique_ids)*0.1) + train_num
train_ids = unique_ids[:train_num]
val_ids = unique_ids[train_num:val_num]
test_ids = unique_ids[val_num:]

In [9]:
train_set = DataFrame()
train_set = disc_inp_data.loc[disc_inp_data['stay_id'].isin(train_ids)]

val_set = DataFrame()
val_set = disc_inp_data.loc[disc_inp_data['stay_id'].isin(val_ids)]

test_set = DataFrame()
test_set = disc_inp_data.loc[disc_inp_data['stay_id'].isin(test_ids)]

In [10]:
# cap values in train and test
caps = pd.read_csv("C:\\Users\\sara1\\Desktop\\Workingdata\\capping_values.csv")
for i in caps.index:
    param = caps.loc[i,'Parameter']
    maxval = caps.loc[i,'limsup']
    minval = caps.loc[i,'liminf']
    train_set[param][train_set[param] >= maxval] = maxval
    train_set[param][train_set[param] <= minval] = minval
    val_set[param][val_set[param] >= maxval] = maxval
    val_set[param][val_set[param] <= minval] = minval
    test_set[param][test_set[param] >= maxval] = maxval
    test_set[param][test_set[param] <= minval] = minval

C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/1322005211.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set[param][train_set[param] >= maxval] = maxval
C:\Users\sara1\Anaconda3\lib\site-packages\pandas\core\generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/1322005211.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_s

In [11]:
binary_fields = ['gender']
norm_fields= ['sofa_score', 'sirs', 'age', 'weight', 'gcs', 'sofa_24hours',
              'hemoglobin', 'chloride', 'calcium', 'potassium', 'sodium',
              'lactate', 'glucose', 'respiratory_rate_total', 'fio2',
              'platelet', 'wbc', 'albumin', 'heart_rate', 'temperature',
              'bicarbonate']
log_fields = ['bilirubin_total', 'creatinine', 'spo2', 'vaso_rate',
              'vaso_amount', 'urineoutput']

In [14]:
# Define a dictionary to map gender strings to numerical representation
#gender_map = {'F': 0, 'M': 1}

# Replace gender values using the dictionary mapping
#train_set[binary_fields] = train_set[binary_fields].replace(gender_map)
#val_set[binary_fields] = val_set[binary_fields].replace(gender_map)
#test_set[binary_fields] = test_set[binary_fields].replace(gender_map)

C:\Users\sara1\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
# normalise binary fields
train_set[binary_fields] = train_set[binary_fields] - 0.5 
val_set[binary_fields] = val_set[binary_fields] - 0.5 
test_set[binary_fields] = test_set[binary_fields] - 0.5 

C:\Users\sara1\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
# normal distn fields
for item in norm_fields:
    av = train_set[item].mean()
    std = train_set[item].std()
    train_set[item] = (train_set[item] - av) / std
    val_set[item] = (val_set[item] - av) / std
    test_set[item] = (test_set[item] - av) / std

C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/163451765.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set[item] = (train_set[item] - av) / std
C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/163451765.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set[item] = (val_set[item] - av) / std
C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/163451765.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [14]:
# log normal fields
train_set[log_fields] = np.log(0.1 + train_set[log_fields])
val_set[log_fields] = np.log(0.1 + val_set[log_fields])
test_set[log_fields] = np.log(0.1 + test_set[log_fields])
for item in log_fields:
    av = train_set[item].mean()
    std = train_set[item].std()
    train_set[item] = (train_set[item] - av) / std
    val_set[item] = (val_set[item] - av) / std
    test_set[item] = (test_set[item] - av) / std

C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/1379520540.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set[item] = (train_set[item] - av) / std
C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/1379520540.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set[item] = (val_set[item] - av) / std
C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/1379520540.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [15]:
train_set.head()

,stay_id,sofa_score,sirs,age,weight,gender,gcs,sofa_24hours,bilirubin_total,bicarbonate,...,creatinine,heart_rate,temperature,spo2,vaso_rate,vaso_amount,urineoutput,died_in_hosp,vaso_input,reward
0,34712382,0.389757,-0.131801,2.066107,-1.620572,-0.5,0.780172,-0.289131,0.086269,1.021089,...,3.023963,-1.249497,0.02012,0.616259,0.213912,0.190777,-0.642081,0,2.0,0
1,34712382,0.389757,-0.131801,2.066107,-1.620572,-0.5,0.780172,-0.289131,0.086269,-0.407944,...,3.023963,-1.249497,0.02012,0.616259,0.213912,0.190777,-0.642081,0,2.0,0
2,34712382,0.389757,-0.131801,2.066107,-1.620572,-0.5,0.780172,-0.289131,0.086269,-1.020386,...,3.023963,-1.249497,0.02012,0.616259,0.213912,0.190777,-0.642081,1,2.0,0
3,34712382,0.389757,-0.131801,2.066107,-1.620572,-0.5,0.780172,-0.289131,0.086269,0.816941,...,3.023963,-1.249497,0.02012,0.616259,0.213912,0.190777,-0.642081,1,2.0,-100
4,30902754,-1.052796,1.139598,1.777622,-0.360729,0.5,-0.818006,-1.546282,0.086269,1.429384,...,-6.893833,0.424134,0.02012,0.693225,0.219767,0.806925,1.962116,1,4.0,0


In [16]:
train_set.to_csv("C:\\Users\\sara1\\Desktop\\Workingdata\\rl_train_set_unscaled.csv",index = False)
val_set.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_set_unscaled.csv', index = False)
test_set.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_test_set_unscaled.csv', index = False)

In [17]:
# scale features to [0,1] in train set, similar in val and test
import copy
scalable_fields = copy.deepcopy(binary_fields)
scalable_fields.extend(norm_fields)
scalable_fields.extend(log_fields)
for col in scalable_fields:
    minimum = min(train_set[col])
    maximum = max(train_set[col])
    train_set[col] = (train_set[col] - minimum)/(maximum-minimum)
    val_set[col] = (val_set[col] - minimum)/(maximum-minimum)
    test_set[col] = (test_set[col] - minimum)/(maximum-minimum)

C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/4234280597.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set[col] = (train_set[col] - minimum)/(maximum-minimum)
C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/4234280597.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set[col] = (val_set[col] - minimum)/(maximum-minimum)
C:\Users\sara1\AppData\Local\Temp/ipykernel_8096/4234280597.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [18]:
train_set.head()

,stay_id,sofa_score,sirs,age,weight,gender,gcs,sofa_24hours,bilirubin_total,bicarbonate,...,creatinine,heart_rate,temperature,spo2,vaso_rate,vaso_amount,urineoutput,died_in_hosp,vaso_input,reward
0,34712382,0.277778,0.75,0.924051,0.091912,0.0,0.916667,0.304348,0.514568,0.8125,...,0.537493,0.223684,0.605949,0.990960,0.359097,0.699997,0.000000,0,2.0,0
1,34712382,0.277778,0.75,0.924051,0.091912,0.0,0.916667,0.304348,0.514568,0.3750,...,0.537493,0.223684,0.605949,0.990960,0.359097,0.699997,0.000000,0,2.0,0
2,34712382,0.277778,0.75,0.924051,0.091912,0.0,0.916667,0.304348,0.514568,0.1875,...,0.537493,0.223684,0.605949,0.990960,0.359097,0.699997,0.000000,1,2.0,0
3,34712382,0.277778,0.75,0.924051,0.091912,0.0,0.916667,0.304348,0.514568,0.7500,...,0.537493,0.223684,0.605949,0.990960,0.359097,0.699997,0.000000,1,2.0,-100
4,30902754,0.000000,1.00,0.873418,0.145221,1.0,0.416667,0.043478,0.514568,0.9375,...,0.118922,0.486842,0.605949,0.993254,0.359400,0.840278,0.644594,1,4.0,0


In [19]:
train_set.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_train_set_scaled.csv',index = False)
val_set.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_set_scaled.csv', index = False)
test_set.to_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_test_set_scaled.csv', index = False)